In [3]:
# Import functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import os
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [4]:
data_dir='salmonella_data/data/'
kmc5_arrays='salmonella_kmc5_arrays/'
kmc7_arrays='salmonella_kmc7_arrays/'
mlst_results = data_dir + 'mlst_results_combined.tsv'
seqsero_results = data_dir + 'seqsero2_results.tsv'
quast_result = data_dir + 'quast_results_combined.tsv'
mlst_df = pd.read_csv(mlst_results, sep='\t', header=None)
seqsero_df = pd.read_csv(seqsero_results, sep='\t')
quast_df = pd.read_csv(quast_result, sep='\t')

In [5]:
# --- PREPROCESSING ---

# Remove very low quality assemblies
quast_df_filt=quast_df[quast_df['N50'] >= 10000]['Assembly']
print(quast_df_filt.shape[0]/quast_df.shape[0]*100, "% of samples retained after N50 filtering")

# Remove samples with no MLST calls
mlst_df_filt = mlst_df[mlst_df.iloc[:, 2] != '-'].iloc[:, [0, 2]]
print(mlst_df_filt.shape[0]/mlst_df.shape[0]*100, "% of samples retained after mlst filtering")

# Basename the first column of mlst_df_filt
mlst_df_filt.iloc[:, 0] = mlst_df_filt.iloc[:, 0].apply(lambda x: x.split('/')[-1].replace('.fna', ''))

# Keep only samples with 50 replicates of the same sequence type
mlst_counts = mlst_df_filt.iloc[:, 1].value_counts()
mlst_df_filt = mlst_df_filt[mlst_df_filt.iloc[:, 1].isin(mlst_counts[mlst_counts >= 100].index)]

# Keep only samples present in both quast and mlst filtered dataframes
common_samples = set(quast_df_filt).intersection(set(mlst_df_filt.iloc[:, 0]))
mlst_df_filt = mlst_df_filt[mlst_df_filt.iloc[:, 0].isin(common_samples)]

print(mlst_df_filt.shape[0]/mlst_df.shape[0]*100, "% of samples retained after merging quast and mlst filtered dataframes")
print(mlst_df_filt.shape[0], "# samples retained after merging quast and mlst filtered dataframes")

# Randomly sample 100 samples from each sequence type
mlst_df_filt = mlst_df_filt.groupby(mlst_df_filt.columns[1]).apply(lambda x: x.sample(100, random_state=42)).reset_index(drop=True)
print(mlst_df_filt.shape[0]/mlst_df.shape[0]*100, "% of samples retained after sampling 50 samples from each sequence type")
print(mlst_df_filt.shape[0], "# samples retained after sampling 50 samples from each sequence type")

# Print number of unique sequence types
print(mlst_df_filt.iloc[:, 1].nunique(), "# unique sequence types")

# Split data into train, validation, and test sets
train_val_df, test_df = train_test_split(mlst_df_filt, test_size=0.2, stratify=mlst_df_filt.iloc[:, 1], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, stratify=train_val_df.iloc[:, 1], random_state=42)
print(train_df.shape[0], "# samples in training set")
print(val_df.shape[0], "# samples in validation set")
print(test_df.shape[0], "# samples in test set")

99.94561379299148 % of samples retained after N50 filtering
98.45705018821108 % of samples retained after mlst filtering
92.41693518679263 % of samples retained after merging quast and mlst filtered dataframes
436524 # samples retained after merging quast and mlst filtered dataframes
5.377459552612303 % of samples retained after sampling 50 samples from each sequence type
25400 # samples retained after sampling 50 samples from each sequence type
254 # unique sequence types
16256 # samples in training set
4064 # samples in validation set
5080 # samples in test set


/tmp/ipykernel_11237/763891927.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mlst_df_filt = mlst_df_filt.groupby(mlst_df_filt.columns[1]).apply(lambda x: x.sample(100, random_state=42)).reset_index(drop=True)


In [6]:
# --- LOAD DATA ---

# Load kmer count arrays
def load_kmer_arrays(df, array_dir, suffix):
    arrays = []
    labels = []
    for idx, row in df.iterrows():
        sample_id = row[0]  # Assuming the first column is the sample ID
        label = row[2]      # Assuming the third column is the label
        array_path = os.path.join(array_dir, f"{sample_id}{suffix}.npy")
        
        if os.path.exists(array_path):
            array = np.load(array_path)
            # Flatten the array
            array = array.flatten()
            arrays.append(array)
            labels.append(label)
        else:
            print(f"Warning: Array file {array_path} not found.")
        
    return np.array(arrays), np.array(labels)

# Load the data
X_train_5, y_train_5 = load_kmer_arrays(train_df, kmc5_arrays, '_k5_k5')
X_val_5, y_val_5 = load_kmer_arrays(val_df, kmc5_arrays, '_k5_k5')
X_test_5, y_test_5 = load_kmer_arrays(test_df, kmc5_arrays, '_k5_k5')

X_train_7, y_train_7 = load_kmer_arrays(train_df, kmc7_arrays, '_k7_k7')
X_val_7, y_val_7 = load_kmer_arrays(val_df, kmc7_arrays, '_k7_k7')
X_test_7, y_test_7 = load_kmer_arrays(test_df, kmc7_arrays, '_k7_k7')

# Print shapes of the loaded data
print(X_train_5.shape, "shape of training data")
print(X_val_5.shape, "shape of validation data")
print(X_test_5.shape, "shape of test data")

# Print shapes of the loaded data
print(X_train_7.shape, "shape of training data")
print(X_val_7.shape, "shape of validation data")
print(X_test_7.shape, "shape of test data")

(16256, 1024) shape of training data
(4064, 1024) shape of validation data
(5080, 1024) shape of test data
(16256, 16384) shape of training data
(4064, 16384) shape of validation data
(5080, 16384) shape of test data


In [19]:
print(y_train_5.shape, "shape of 5mer training ground truth")
print(y_train_7.shape, "shape of 7mer training ground truth")
print(y_train_5[0:3])
print(len(np.unique(y_train_5)))

(16256,) shape of 5mer training ground truth
(16256,) shape of 7mer training ground truth
['128' '216' '814']
254


In [8]:
# --- MODELS ---

# --- LOGISTIC REGRESSION MODEL ---
def train_logistic_regression(X_train, y_train, X_val, y_val, X_test, y_test):
    # Create and train logistic regression model with standard scaler
    model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
    model.fit(X_train, y_train)  # Train the model on the training set
    y_val_pred = model.predict(X_val)  # Predict on the validation set

    # Print classification report for validation set
    print("Validation Set Classification Report:")
    print(classification_report(y_val, y_val_pred))

    y_test_pred = model.predict(X_test)  # Predict on the test set

    # Print classification report for test set
    print("Test Set Classification Report:")
    print(classification_report(y_test, y_test_pred))

# --- XGBOOST MODEL ---

def train_xgboost(X_train, y_train, X_val, y_val, X_test, y_test):
    # Encode labels if they are categorical
    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)  # Fit on training labels
    y_val_encoded = le.transform(y_val)           # Transform validation labels
    y_test_encoded = le.transform(y_test)         # Transform test labels
    

    # Create and fit the model
    xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, device="cuda")
    xgb_model.fit(X_train,y_train_encoded)

    # Predictions
    y_val_pred_xgb = xgb_model.predict(X_val)
    print("XGBoost Validation Set Classification Report:")
    print(classification_report(y_val_encoded, y_val_pred_xgb))

    y_test_pred_xgb = xgb_model.predict(X_test)
    print("XGBoost Test Set Classification Report:")
    print(classification_report(y_test_encoded, y_test_pred_xgb))

In [11]:
train_logistic_regression(X_train_5, y_train_5, X_val_5, y_val_5, X_test_5, y_test_5)
train_logistic_regression(X_train_7, y_train_7, X_val_7, y_val_7, X_test_7, y_test_7)

Validation Set Classification Report:
              precision    recall  f1-score   support

           1       1.00      0.94      0.97        16
          10       1.00      1.00      1.00        16
         101       1.00      1.00      1.00        16
          11       0.87      0.81      0.84        16
         112       0.85      0.69      0.76        16
         118       0.87      0.81      0.84        16
        1208       1.00      1.00      1.00        16
        1256       1.00      1.00      1.00        16
         126       1.00      1.00      1.00        16
         127       1.00      0.94      0.97        16
         128       0.73      1.00      0.84        16
         129       0.65      0.94      0.77        16
        1291       1.00      1.00      1.00        16
        1298       1.00      1.00      1.00        16
          13       1.00      1.00      1.00        16
        1316       1.00      1.00      1.00        16
         132       0.89      1.00      0.94

In [9]:
train_xgboost(X_train_5, y_train_5, X_val_5, y_val_5, X_test_5, y_test_5)
train_xgboost(X_train_7, y_train_7, X_val_7, y_val_7, X_test_7, y_test_7)

/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/xgboost/training.py:183: UserWarning: [16:24:06] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758007733415/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/xgboost/core.py:705: UserWarning: [16:28:03] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758007733415/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


XGBoost Validation Set Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.62      0.69        16
           1       1.00      0.88      0.93        16
           2       0.94      0.94      0.94        16
           3       0.67      0.50      0.57        16
           4       0.44      0.50      0.47        16
           5       0.50      0.50      0.50        16
           6       1.00      1.00      1.00        16
           7       1.00      0.94      0.97        16
           8       0.76      0.81      0.79        16
           9       0.80      0.75      0.77        16
          10       0.93      0.81      0.87        16
          11       0.67      0.88      0.76        16
          12       0.93      0.88      0.90        16
          13       0.93      0.81      0.87        16
          14       0.71      0.75      0.73        16
          15       0.87      0.81      0.84        16
          16       0.71      0.94  

/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/xgboost/training.py:183: UserWarning: [16:28:18] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758007733415/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Validation Set Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.88      0.93        16
           1       0.94      0.94      0.94        16
           2       1.00      1.00      1.00        16
           3       0.93      0.81      0.87        16
           4       0.81      0.81      0.81        16
           5       0.86      0.75      0.80        16
           6       0.89      1.00      0.94        16
           7       0.84      1.00      0.91        16
           8       0.80      1.00      0.89        16
           9       1.00      1.00      1.00        16
          10       0.84      1.00      0.91        16
          11       0.93      0.81      0.87        16
          12       1.00      0.94      0.97        16
          13       0.94      1.00      0.97        16
          14       1.00      0.88      0.93        16
          15       1.00      0.94      0.97        16
          16       0.89      1.00  

In [85]:
# Generate confusion matrix for test set
# cm = confusion_matrix(y_test, y_test_pred)
# plt.figure(figsize=(10, 8))
# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.colorbar()
# tick_marks = np.arange(len(set(y_test)))
# plt.xticks(tick_marks, set(y_test), rotation=45)
# plt.yticks(tick_marks, set(y_test))
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.tight_layout()
# plt.show()

In [75]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
from torchmetrics import F1Score

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

import optuna
from optuna.integration import PyTorchLightningPruningCallback

torch.set_float32_matmul_precision('high')

In [ ]:
def accuracy(preds, y):
    return (preds == y).float().mean()

In [84]:
def F1(out_dim,preds,y):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Initialize F1Score for binary classification
    f1 = F1Score(task="multiclass", num_classes=out_dim, average='macro').to(device)

    # Calculate F1 score
    return f1(preds, y)

In [82]:
class PanspaceCNN(pl.LightningModule):
    def __init__(self, input_size, out_dim, learning_rate):
        """
        Initializes the PyTorch model for Panspace.

        Args:
            input_size (int): The size of the square CGR image (e.g., 64 for k=6).
            out_dim (int): The size of the output vector.
        """
        super(PanspaceCNN, self).__init__()
        
        # Convolutional layers
        # The number of filters and kernel sizes are example values and can be tuned.
        self.save_hyperparameters()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        # Calculate the size of the flattened tensor after convolutions and pooling.
        # This assumes a 1xinput_size x input_size input image.
        final_conv_size = input_size // (2**3)  # After 3 pooling layers
        self.fc1 = nn.Linear(128 * final_conv_size * final_conv_size, 512)
        self.fc2 = nn.Linear(512, out_dim)

        # Batch normalization and activation functions
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu = nn.ReLU()
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        """
        Defines the forward pass of the model.

        Args:
            x (torch.Tensor): A tensor representing the CGR image of shape (batch_size, 1, H, W).

        Returns:
            torch.Tensor: A tensor with predicted probabilities for each class of shape (batch_size, num_classes)
        """
        # Apply convolutions, batch normalization, and pooling
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        
        # Flatten the tensor
        x = torch.flatten(x, 1)

        # Apply fully connected layers
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        # Apply softmax
        # x = self.softmax(x)
        
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, torch.argmax(y,dim=1))
        f1 = F1(self.hparams.out_dim,preds,torch.argmax(y,dim=1))
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        self.log('val_f1', f1, prog_bar=True)
        return {'val_loss': loss, 'val_acc': acc, 'val_f1': f1}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, torch.argmax(y,dim=1))
        f1 = F1(self.hparams.out_dim,preds,torch.argmax(y,dim=1))
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        self.log('test_f1', f1, prog_bar=True)
        return {'test_loss': loss, 'test_acc': acc, 'test_f1': f1}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

In [73]:
# ----------------------------------
# scale classes for one-hot encoding
# ----------------------------------

def scale(y,classes):
    y_scaled = []
    for label in y:
        index = np.where(classes == label)
        y_scaled.append(index[0][0])
    y_scale = np.array(y_scaled)
    return y_scale

def prepare_data(X_train,X_val,X_test,y_train,y_val,y_test,size=32,out_dim=254):
    y_train = y_train.astype(int)
    y_val = y_val.astype(int)
    y_test = y_test.astype(int)
    classes = np.sort(np.unique(y_train))

    y_train_scale = scale(y_train,classes)
    y_val_scale = scale(y_val,classes)
    y_test_scale = scale(y_test,classes)

    # -----------------------------
    # Convert data to torch tensors
    # -----------------------------
    X_train_tensor = torch.tensor(X_train.reshape(-1,size,size)).unsqueeze(1).float()  # (N, 1, size, size)
    X_val_tensor = torch.tensor(X_val.reshape(-1,size,size)).unsqueeze(1).float()
    X_test_tensor = torch.tensor(X_test.reshape(-1,size,size)).unsqueeze(1).float()
    y_train_tensor = torch.tensor(y_train_scale)
    y_val_tensor = torch.tensor(y_val_scale)
    y_test_tensor = torch.tensor(y_test_scale)

    # -----------------------------
    # Encode labels to categorical
    # -----------------------------
    y_train_enc = F.one_hot(y_train_tensor, num_classes = out_dim).float()
    y_val_enc = F.one_hot(y_val_tensor, num_classes = out_dim).float()
    y_test_enc = F.one_hot(y_test_tensor, num_classes= out_dim).float()

    # -----------------------------
    # Create PyTorch Datasets and Loaders
    # -----------------------------
    train_dataset = TensorDataset(X_train_tensor, y_train_enc)
    val_dataset = TensorDataset(X_val_tensor, y_val_enc)
    test_dataset = TensorDataset(X_test_tensor, y_test_enc)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)
    return train_loader,val_loader,test_loader

In [94]:
# Define the objective function for Optuna
def objective(trial):
    # Set the hyperparameters to optimize
    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-2, log=True)
    
    # Create the model with trial hyperparameters
    model = PanspaceCNN(
        input_size=128,
        out_dim=254,
        learning_rate=learning_rate
    )
    
    # Early stopping callback
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        patience=5,
        verbose=False,
        mode='min'
    )
    
    # Optuna pruning callback
    pruning_callback = PyTorchLightningPruningCallback(trial, monitor='val_loss')
    
    # Logger
    logger = TensorBoardLogger(save_dir=os.getcwd(), name=f"optuna_logs/trial_{trial.number}")
    
    # Create trainer
    trainer = pl.Trainer(
        max_epochs=50,
        callbacks=[early_stop_callback, pruning_callback],
        logger=logger,
        enable_progress_bar=False,  
        enable_model_summary=False  
    )
    
    # Preparing the data
    train_loader, val_loader, test_loader = prepare_data(X_train_7,X_val_7,X_test_7,y_train_7,y_val_7,y_test_7,128,254)
    
    # Training the model
    trainer.fit(model, train_loader, val_loader)
    
    # Final validation loss
    return trainer.callback_metrics['val_loss'].item()

In [59]:
def run_optimization(n_trials=50):
    pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
    study = optuna.create_study(direction='minimize', pruner=pruner)
    study.optimize(objective, n_trials=n_trials)
    
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
        
    return study

In [72]:
def test_best_model(study,X_train,X_val,X_test,y_train,y_val,y_test,size,num_classes):
    # Getting the best hyperparameters
    best_params = study.best_trial.params
    
    # Creating the model with the best hyperparameters
    model = PanspaceCNN(
        input_size=size,
        out_dim=num_classes,
        learning_rate=best_params['learning_rate']
    )
    
    # Creating trainer instance
    trainer = pl.Trainer(max_epochs=150)
    
    # Preparing the data
    train_loader, val_loader, test_loader = prepare_data(X_train,X_val,X_test,y_train,y_val,y_test,size,num_classes)
    
    # Training the model with the best hyperparameters
    trainer.fit(model, train_loader, val_loader)
    
    # Testing the model with the test data
    results = trainer.test(model, dataloaders=test_loader)
    return results

In [95]:
study = run_optimization(n_trials=25)

[I 2025-11-03 15:59:39,018] A new study created in memory with name: no-name-4bcbb982-1aba-4e98-8ca7-32af0f9574a8
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_epochs=50` reached.
[I 2025-11-03 16:16:09,297] Trial 0 finished with value: 5.537333011627197 and parameters: {'learning_rate': 0.000343417155197835}. Best is trial 0 with value: 5.537333011627197.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU availab

Epoch 10:  51%|█████     | 129/254 [34:57<33:52,  0.06it/s, v_num=8, train_loss=5.540, val_loss=5.540, val_acc=0.00394, val_f1=0.000135]


`Trainer.fit` stopped: `max_epochs=50` reached.
[I 2025-11-03 16:32:50,632] Trial 1 finished with value: 5.463733196258545 and parameters: {'learning_rate': 2.073647366715919e-07}. Best is trial 1 with value: 5.463733196258545.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[I 2025-11-03 16:49:04,777] Trial 2 finished with value: 5.537347316741943 and parameters: {'learning_rate': 0.0004792266424093759}. Best is trial 1 with value: 5.463733196258545.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cud

Best trial:
  Value: 5.097236633300781
  Params: 
    learning_rate: 2.7006153026513106e-06


In [ ]:
# Best trial:
#   Value: 0.973054051399231
#   Params: 
#     learning_rate: 4.786642076991138e-05

# Visualize the results
try:
    # Plot optimization history
    optuna.visualization.plot_optimization_history(study)
    
    # Plot parameter importances
    optuna.visualization.plot_param_importances(study)
    
    # Plot parallel coordinate plot
    optuna.visualization.plot_parallel_coordinate(study)
except ImportError:
    print("Visualization requires plotly. Install with: pip install plotly")
  
  # Test the best model
results = test_best_model(study,X_train_5,X_val_5,X_test_5,y_train_5,y_val_5,y_test_5,32,254)
print(f"Test results with best hyperparameters: {results}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params | Mode 
----------------------------------------------
0 | conv1 | Conv2d      | 320    | train
1 | conv2 | Conv2d      | 18.5 K | train
2 | conv3 | Conv2d      | 73.9 K | train
3 | pool  | MaxPool2d   | 0      | train
4 | fc1   | Linear      | 1.0 M  | train
5 | fc2   | Linear      | 130 K  | train
6 | bn1   | BatchNorm2d | 64     | train
7 | bn2   | BatchNorm2d | 128    | train
8 | bn3   | BatchNorm2d | 256    | train
9 | relu  | ReLU        | 0      | train
----------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.090     

/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



Epoch 149: 100%|██████████| 254/254 [00:02<00:00, 91.20it/s, v_num=7, train_loss=0.255, val_loss=0.373, val_acc=0.894, val_f1=0.821]    

`Trainer.fit` stopped: `max_epochs=150` reached.


Epoch 149: 100%|██████████| 254/254 [00:03<00:00, 84.08it/s, v_num=7, train_loss=0.255, val_loss=0.373, val_acc=0.894, val_f1=0.821]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.




Testing DataLoader 0: 100%|██████████| 80/80 [00:00<00:00, 130.49it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9007874131202698
         test_f1            0.8333085775375366
        test_loss           0.3248129189014435
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test results with best hyperparameters: [{'test_loss': 0.3248129189014435, 'test_acc': 0.9007874131202698, 'test_f1': 0.8333085775375366}]


In [96]:
# Best trial:
#   Value: 5.097236633300781
#   Params: 
#     learning_rate: 2.7006153026513106e-06
# Visualize the results
try:
    # Plot optimization history
    optuna.visualization.plot_optimization_history(study)
    
    # Plot parameter importances
    optuna.visualization.plot_param_importances(study)
    
    # Plot parallel coordinate plot
    optuna.visualization.plot_parallel_coordinate(study)
except ImportError:
    print("Visualization requires plotly. Install with: pip install plotly")
  
  # Test the best model
results = test_best_model(study,X_train_7,X_val_7,X_test_7,y_train_7,y_val_7,y_test_7,128,254)
print(f"Test results with best hyperparameters: {results}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params | Mode 
----------------------------------------------
0 | conv1 | Conv2d      | 320    | train
1 | conv2 | Conv2d      | 18.5 K | train
2 | conv3 | Conv2d      | 73.9 K | train
3 | pool  | MaxPool2d   | 0      | train
4 | fc1   | Linear      | 16.8 M | train
5 | fc2   | Linear      | 130 K  | train
6 | bn1   | BatchNorm2d | 64     | train
7 | bn2   | BatchNorm2d | 128    | train
8 | bn3   | BatchNorm2d | 256    | train
9 | relu  | ReLU        | 0      | train
----------------------------------------------
17.0 M    Trainable params
0         Non-trainable params
17.0 M    Total params
68.005    

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



Epoch 149: 100%|██████████| 254/254 [00:17<00:00, 14.71it/s, v_num=9, train_loss=4.450, val_loss=4.470, val_acc=0.225, val_f1=0.139]    

`Trainer.fit` stopped: `max_epochs=150` reached.


Epoch 149: 100%|██████████| 254/254 [00:20<00:00, 12.60it/s, v_num=9, train_loss=4.450, val_loss=4.470, val_acc=0.225, val_f1=0.139]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kmorin/anaconda3/envs/Ethan_MLB_final/lib/python3.14/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: PossibleUserWarning:

The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.




Testing DataLoader 0: 100%|██████████| 80/80 [00:02<00:00, 35.18it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.2287401556968689
         test_f1            0.1434086114168167
        test_loss           4.4602813720703125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test results with best hyperparameters: [{'test_loss': 4.4602813720703125, 'test_acc': 0.2287401556968689, 'test_f1': 0.1434086114168167}]
